# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hamilton,39.1834,-84.5333,25.61,77,38,2.68,US,1722641057
1,1,adamstown,-25.0660,-130.1015,20.28,77,23,2.94,PN,1722641424
2,2,delmas,18.5447,-72.3028,31.40,55,40,7.20,HT,1722641425
3,3,qaqortoq,60.7167,-46.0333,7.51,80,0,0.43,GL,1722641427
4,4,port-aux-francais,-49.3500,70.2167,1.86,69,78,12.42,TF,1722641428


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
weather_df = city_data_df
weather_df = weather_df.loc[(weather_df["Max Temp"] < 27) & (weather_df["Max Temp"] > 21)]
weather_df = weather_df.loc[(weather_df["Wind Speed"] < 4.5)]
weather_df = weather_df.loc[(weather_df["Cloudiness"] == 0)]

# Drop any rows with null values
weather_df = weather_df.dropna()

# Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
164,164,moroni,-11.7022,43.2551,23.74,60,0,2.62,KM,1722641610
170,170,mikhaylovsk,45.1283,42.0256,22.12,40,0,3.00,RU,1722641616
229,229,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1722641390
235,235,aibak,36.2647,68.0155,25.58,19,0,2.64,AF,1722641390
334,334,saranac lake,44.3295,-74.1313,25.18,78,0,0.00,US,1722641809
355,355,demre,36.2537,29.9777,25.70,63,0,3.39,TR,1722641832
395,395,cambridge,43.3601,-80.3127,25.66,88,0,0.00,CA,1722641865
410,410,archangelos,36.2149,28.1149,24.06,71,0,3.57,GR,1722641895
467,467,san vicente,-26.6167,-54.1333,22.81,43,0,4.37,AR,1722641961
552,552,isola di capo rizzuto,38.9600,17.0933,26.80,56,0,1.97,IT,1722642078


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
164,moroni,KM,-11.7022,43.2551,60,
170,mikhaylovsk,RU,45.1283,42.0256,40,
229,remire-montjoly,GF,4.9167,-52.2667,94,
235,aibak,AF,36.2647,68.0155,19,
334,saranac lake,US,44.3295,-74.1313,78,
355,demre,TR,36.2537,29.9777,63,
395,cambridge,CA,43.3601,-80.3127,88,
410,archangelos,GR,36.2149,28.1149,71,
467,san vicente,AR,-26.6167,-54.1333,43,
552,isola di capo rizzuto,IT,38.9600,17.0933,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
    
    

}

# Print a message to follow up the hotel search
#print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(name_address.url)

    # Convert the API response to JSON format
    #name_address = name_address.json()
    

https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ff348ef1ed81442ab6814fdfb229ed56&filter=circle%3A43.2551%2C-11.7022%2C10000&bias=proximity%3A43.2551%2C-11.7022
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ff348ef1ed81442ab6814fdfb229ed56&filter=circle%3A42.0256%2C45.1283%2C10000&bias=proximity%3A42.0256%2C45.1283
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ff348ef1ed81442ab6814fdfb229ed56&filter=circle%3A-52.2667%2C4.9167%2C10000&bias=proximity%3A-52.2667%2C4.9167
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ff348ef1ed81442ab6814fdfb229ed56&filter=circle%3A68.0155%2C36.2647%2C10000&bias=proximity%3A68.0155%2C36.2647
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ff348ef1ed81442ab6814fdfb229ed56&filter=circle%3A-74.1313%2C44.3295%2C10000&bias=proximity%3A-74.1313%2C44.3295
https://api.geoapify.com/v2/places?categories=accommodation.hotel&apiKey=ff348ef1

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
    

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
moroni - nearest hotel: Ville de Moroni
mikhaylovsk - nearest hotel: Усадьба
remire-montjoly - nearest hotel: Complexe Belova
aibak - nearest hotel: No hotel found
saranac lake - nearest hotel: Hotel Saranac
demre - nearest hotel: Hotel Kekova
cambridge - nearest hotel: Travelodge
archangelos - nearest hotel: Semina
san vicente - nearest hotel: Los Lapachos
isola di capo rizzuto - nearest hotel: No hotel found
torres - nearest hotel: Hotel Carioca
meknes - nearest hotel: Hotel Swani
waddan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
164,moroni,KM,-11.7022,43.2551,60,Ville de Moroni
170,mikhaylovsk,RU,45.1283,42.0256,40,Усадьба
229,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
235,aibak,AF,36.2647,68.0155,19,No hotel found
334,saranac lake,US,44.3295,-74.1313,78,Hotel Saranac
355,demre,TR,36.2537,29.9777,63,Hotel Kekova
395,cambridge,CA,43.3601,-80.3127,88,Travelodge
410,archangelos,GR,36.2149,28.1149,71,Semina
467,san vicente,AR,-26.6167,-54.1333,43,Los Lapachos
552,isola di capo rizzuto,IT,38.9600,17.0933,56,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    tiles = "OSM", 
    frame_width = 900, 
    frame_height = 700, 
    color = "City", 
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)